In [1]:
import sys
import os

# 현재 Jupyter Notebook이 실행 중인 디렉토리를 기준으로 상위 경로 추가
sys.path.append(os.path.abspath(".."))

# 모듈 불러오기
from dart_tool.insider_trade_tool import DARTExecutiveShareholdingAPI, DARTMajorStockReportAPI

# 기업 코드

In [ ]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
# API 키 입력
DART_API_KEY = os.getenv("DART_API_KEY")
# 고유번호 요청 URL
url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={DART_API_KEY}"

# 데이터 요청
response = requests.get(url)

if response.status_code == 200:
    # zip 파일 열기
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # 압축 해제 후 corpCode.xml 읽기
        with z.open('CORPCODE.xml') as xml_file:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            
            # 데이터를 담을 리스트
            corp_list = []

            # XML 데이터 파싱
            for corp in root.findall('list'):
                corp_code = corp.findtext('corp_code')
                corp_name = corp.findtext('corp_name')
                corp_eng_name = corp.findtext('corp_eng_name')
                stock_code = corp.findtext('stock_code')
                modify_date = corp.findtext('modify_date')
                
                corp_list.append({
                    "corp_code": corp_code,
                    "corp_name": corp_name,
                    "corp_eng_name": corp_eng_name,
                    "stock_code": stock_code,
                    "modify_date": modify_date
                })

            # DataFrame으로 변환
            corp_df = pd.DataFrame(corp_list)

            # 결과 출력
            print(corp_df.head())

else:
    print(f"요청 실패: 상태 코드 {response.status_code}")


# 임원 소유 보고서 테스트

In [2]:
import pandas as pd

def test_case_1_no_date():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", limit=5
    )
    assert isinstance(df, pd.DataFrame)
    assert not df.empty
    print("test_case_1_no_date 통과")
    print(df)

def test_case_2_start_only():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", start_date="2025-01-01", limit=5
    )
    assert all(df["rcept_dt"] >= pd.to_datetime("2025-01-01"))
    print("test_case_2_start_only 통과")
    print(df)

def test_case_3_end_only():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", end_date="2025-12-31", limit=5
    )
    assert all(df["rcept_dt"] <= pd.to_datetime("2025-12-31"))
    print("test_case_3_end_only 통과")
    print(df)

def test_case_4_start_and_end():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", start_date="2024-12-01", end_date="2025-12-31", limit=5
    )
    assert all((df["rcept_dt"] >= pd.to_datetime("2024-12-01")) &
               (df["rcept_dt"] <= pd.to_datetime("2025-12-31")))
    print("test_case_4_start_and_end 통과")
    print(df)

def test_case_5_reference_only():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", reference_date="2025-03-22", limit=5
    )
    assert not df.empty
    print("test_case_5_reference_only 통과")
    print(df)

def test_case_6_invalid_stock_code():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="000000", limit=5
    )
    assert df.empty
    print("test_case_6_invalid_stock_code 통과 (빈 결과 확인됨)")

def test_case_7_all_dates():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930",
        start_date="2024-12-01",
        end_date="2025-12-31",
        reference_date="2025-03-12",
        limit=5
    )
    # reference_date는 무시됨
    assert all((df["rcept_dt"] >= pd.to_datetime("2024-12-01")) &
               (df["rcept_dt"] <= pd.to_datetime("2025-12-31")))
    print("test_case_7_all_dates 통과")
    print(df)

def test_case_8_corp_name_with_spaces():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        corp_name=" 삼성 전자 ", reference_date="2025-03-12", limit=5
    )
    assert not df.empty
    print("test_case_8_corp_name_with_spaces 통과")
    print(df)

def test_case_9_both_inputs_provided():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930", corp_name="삼성전자", reference_date="2025-03-11", limit=5
    )
    assert not df.empty
    print("test_case_9_both_inputs_provided 통과")
    print(df)

if __name__ == "__main__":
    test_case_1_no_date()
    test_case_2_start_only()
    test_case_3_end_only()
    test_case_4_start_and_end()
    test_case_5_reference_only()
    test_case_6_invalid_stock_code()
    test_case_7_all_dates()
    test_case_8_corp_name_with_spaces()
    test_case_9_both_inputs_provided()


test_case_1_no_date 통과
    rcept_dt corp_code corp_name          repror isu_exctv_rgist_at  \
0 2025-03-21  00126380      삼성전자             이혁재               등기임원   
1 2025-03-19  00126380      삼성전자  SONG SANG CHUL              비등기임원   
2 2025-03-14  00126380      삼성전자  SONG SANG CHUL              비등기임원   
3 2025-03-11  00126380      삼성전자             임전식              비등기임원   
4 2025-02-26  00126380      삼성전자             윤성호              비등기임원   

  isu_exctv_ofcps isu_main_shrholdr sp_stock_lmp_cnt sp_stock_lmp_irds_cnt  \
0            사외이사                 -              200                   200   
1              상무                 -            1,500                -1,000   
2              상무                 -            2,500                 2,000   
3              상무                 -              280                    80   
4              상무                 -              200                   100   

  sp_stock_lmp_rate sp_stock_lmp_irds_rate  
0              0.00                 

In [3]:
def test_case_7_all_dates():
    df = DARTExecutiveShareholdingAPI._get_executive_shareholding(
        stock_code="005930",
        start_date="2024-12-01",
        end_date="2025-12-31",
        reference_date="2025-03-12",
        limit=5
    )
    return df

In [6]:
df = test_case_7_all_dates()

# 대량 보유 상황 보고서 API

In [1]:
import sys
import os
import pandas as pd
# 현재 Jupyter Notebook이 실행 중인 디렉토리를 기준으로 상위 경로 추가
sys.path.append(os.path.abspath(".."))

# 모듈 불러오기
from dart_tool.insider_trade_tool import DARTExecutiveShareholdingAPI, DARTMajorStockReportAPI
api = DARTMajorStockReportAPI()

In [22]:

## 기본
def test_majorstock_basic():
    df = api._get_major_stock_reports(corp_name="삼성전자")
    assert not df.empty
    assert "rcept_dt" in df.columns


## 날짜 필터
def test_majorstock_start_date_only():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        start_date="2024-11-01"
    )
    assert not df.empty
    assert df["rcept_dt"].min() >= pd.to_datetime("2024-11-01")


def test_majorstock_end_date_only():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        end_date="2024-12-31"
    )
    assert not df.empty
    assert df["rcept_dt"].max() <= pd.to_datetime("2024-12-31")


def test_majorstock_reference_date():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        reference_date="2024-12-01"
    )
    assert not df.empty
    assert df["rcept_dt"].max() <= pd.to_datetime("2024-12-01")


## 수치 필터
def test_majorstock_min_ratio():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        min_ratio=20.5
    )
    assert not df.empty
    assert df["stkrt"].astype(float).min() >= 20.5


def test_majorstock_min_ratio_change():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        min_ratio_change=0.01
    )
    assert not df.empty
    assert df["stkrt_irds"].astype(float).min() >= 0.01


def test_majorstock_min_share_count():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        min_share_count=1_200_000_000
    )
    assert not df.empty
    assert df["stkqy"].min() >= 1_200_000_000


def test_majorstock_min_share_change():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        min_share_change=100
    )
    assert not df.empty
    assert df["stkqy_irds"].min() >= 100




def test_majorstock_max_share_change():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        max_share_change=-1
    )
    assert not df.empty
    assert df["stkqy_irds"].max() <= -1



## 날짜 + 수치 필터 조합
def test_majorstock_date_and_share_filter():
    df = api._get_major_stock_reports(
        corp_name="삼성전자",
        start_date="2021-01-01",
        min_share_change=100
    )
    assert not df.empty
    assert df["stkqy_irds"].min() >= 100



In [ ]:
def main():
    test_majorstock_basic()

    test_majorstock_start_date_only()

    test_majorstock_end_date_only()

    test_majorstock_reference_date()

    test_majorstock_min_ratio()

    test_majorstock_min_ratio_change()

    test_majorstock_min_share_count()

    test_majorstock_min_share_change()

    test_majorstock_max_share_change()

    test_majorstock_date_and_share_filter()
    print("모든 테스트가 통과.")

main()


1
1
1
1
1
1
2
2
3
모든 테스트가 통과되었습니다.


In [10]:
df = api._get_major_stock_reports(corp_name="삼성전자")
print(df[["rcept_dt", "stkqy_irds"]].sort_values(by="stkqy_irds", ascending=False))


     rcept_dt  stkqy_irds
8  2024-10-25       57566
15 2024-05-17       14495
9  2024-10-25        6317
0  2025-03-21      -10682
10 2024-10-25      -11113
3  2025-01-24      -21972
4  2025-01-24      -38791
18 2024-04-19      -47314
12 2024-07-26      -55084
16 2024-05-03      -62823
1  2025-03-07      -77470
5  2024-12-20      -91303
11 2024-10-25     -117240
14 2024-05-24     -205875
6  2024-11-15     -259744
17 2024-04-26     -309948
13 2024-07-19     -347788
7  2024-10-25     -348407
2  2025-02-14    -4963824
19 2024-04-12    -5332668
